In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from geopy.geocoders import Nominatim
from IPython.display import display
import time

# Inicializar el geocodificador
geolocator = Nominatim(user_agent="mi_aplicacion_mapa")

# Cargar el archivo SHP de municipios
shapefile_municipios = "2023_1_19_A/2023_1_19_A.shp"  # Cambia a la ruta de tu archivo
gdf_municipios = gpd.read_file(shapefile_municipios)

# Cargar el archivo SHP de colonias
shapefile_colonias = "Colonias/Colonias.shp"  # Cambia a la ruta de tu archivo
gdf_colonias = gpd.read_file(shapefile_colonias)

# Cargar los DataFrames con los puntajes
df_municipios = pd.read_csv('df_con_colonias_actualizadas11.csv')
df_colonias = pd.read_csv('df_con_colonias_actualizadas.csv')

# Excluir los municipios "Monte Morelos" y "Saltillo"
df_municipios = df_municipios[~df_municipios['municipio'].isin(['monte morelos', 'saltillo'])]

# Filtrar los municipios y calcular el puntaje promedio por municipio
cve_mun_values = {
    '006': 'apodaca', 
    '019': 'san pedro garza garcía', 
    '021': 'escobedo', 
    '026': 'guadalupe', 
    '039': 'monterrey', 
    '046': 'san nicolás', 
    '048': 'santa catarina', 
    '049': 'santiago'
}

promedio_puntajes_municipios = {}
for cve, municipio in cve_mun_values.items():
    filas_municipio = df_municipios[df_municipios['municipio'].str.lower() == municipio.lower()]
    if not filas_municipio.empty:
        promedio_puntajes_municipios[cve] = np.mean(filas_municipio['Puntaje'])
    else:
        promedio_puntajes_municipios[cve] = None

# Asignar los puntajes promedios a los códigos CVE_MUN correspondientes
gdf_municipios['Puntaje'] = gdf_municipios['CVE_MUN'].map(promedio_puntajes_municipios)

# Configurar el colormap para municipios
norm_municipios = Normalize(vmin=-2, vmax=2)  # Ajusta estos valores según tus datos
cmap_municipios = cm.get_cmap('RdYlGn')

# Crear el mapa centrado en Monterrey
mapa = folium.Map(location=[25.6866, -100.3161], zoom_start=13)

# Añadir los polígonos de municipios al mapa
for _, row in gdf_municipios.iterrows():
    puntaje = row['Puntaje']
    if puntaje is not None:
        color = cmap_municipios(norm_municipios(puntaje))[:3]
        color = [int(c * 255) for c in color]
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x, color=color: {
                'fillColor': f'rgba({color[0]}, {color[1]}, {color[2]}, 0.6)',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            }
        ).add_to(mapa)

# Añadir los polígonos de colonias al mapa
# Convertir la columna 'colonia' a mayúsculas en df_colonias
df_colonias['colonia'] = df_colonias['colonia'].str.upper()

# Filtrar las colonias y hacer merge con los puntajes
gdf_colonias = gdf_colonias.merge(df_colonias, left_on='SETT_NAME', right_on='colonia', how='left')

# Configurar el colormap para colonias
norm_colonias = Normalize(vmin=df_colonias['Puntaje'].min(), vmax=df_colonias['Puntaje'].max())
cmap_colonias = cm.get_cmap('RdYlGn')

# Añadir los polígonos de colonias al mapa
for _, row in gdf_colonias.iterrows():
    puntaje = row['Puntaje']
    if puntaje is not None:
        color = cmap_colonias(norm_colonias(puntaje))[:3]
        color = [int(c * 255) for c in color]
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x, color=color: {
                'fillColor': f'rgba({color[0]}, {color[1]}, {color[2]}, 0.6)',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            }
        ).add_to(mapa)

# Añadir marcadores para las avenidas
for avenida in [
    "Calle Pirineos, Monterrey, Mexico",
    "Avenida Eugenio Garza Sada, Monterrey, Mexico",
    "Avenida Lázaro Cárdenas, Monterrey, Mexico",
    "Calzada del Valle, San Pedro Garza García, Mexico",
]:
    try:
        location = geolocator.geocode(avenida)
        time.sleep(1)  # Espera de 1 segundo entre solicitudes
        if location:
            folium.Marker(
                location=[location.latitude, location.longitude],
                popup=avenida.split(",")[0],
                icon=folium.Icon(icon="cloud"),
            ).add_to(mapa)
    except Exception as e:
        print(f"Error al intentar obtener las coordenadas para {avenida}: {e}")

# Mostrar el mapa en Jupyter
display(mapa)
